In [1]:
%%capture
# install ngrok linux version using the following command or you can get the
! pip install flask_ngrok

# latest version from its official website- https://dashboard.ngrok.com/get-started/setup
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz

# extract the downloaded file using the following command
!tar -xvf /content/ngrok-stable-linux-amd64.tgz

# paste your AuthToken here and execute this command
!./ngrok authtoken 2N35FLCpygD7HxTC7YcXynkV1Rb_58A3F9q4nPubdmhTdcao6

!gdown --id 17um-mG_4stndk3SPy5XsHSVzn4rYyTd-

!pip install PyArabic

In [2]:
from flask import Flask, request, jsonify, render_template, render_template_string, redirect, url_for
import joblib
import pyarabic.araby as araby
import re
from flask_ngrok import run_with_ngrok
import os
os.mkdir('templates')

In [3]:
index_html = """<!DOCTYPE html>
<html>
  <head>
    <title>Text Classification Demo</title>
    <style>
      /* Center the content vertically and horizontally */
      body {
        display: flex;
        align-items: center;
        justify-content: center;
        height: 100vh;
        background-color: #15202b;
        color: #fff;
        font-family: Arial, sans-serif;
      }
      /* Style the form container */
      .form-container {
        max-width: 400px;
        padding: 20px;
        border-radius: 16px;
        background-color: #192734;
        box-shadow: 0 0 16px rgba(0, 0, 0, 0.3);
      }
      /* Style the form elements */
      label {
        display: block;
        margin-bottom: 8px;
        font-weight: bold;
        font-size: 16px;
      }
      input[type="text"], textarea {
        width: 100%;
        padding: 12px 20px;
        margin: 8px 0;
        box-sizing: border-box;
        border: none;
        border-radius: 8px;
        background-color: #1c2938;
        color: #fff;
        font-size: 16px;
      }
      input[type="text"]::placeholder, textarea::placeholder {
        color: #8899a6;
        opacity: 1;
      }
      input[type="submit"] {
        background-color: #1da1f2;
        color: #fff;
        border: none;
        border-radius: 8px;
        padding: 12px 20px;
        margin-top: 16px;
        cursor: pointer;
        font-size: 16px;
        font-weight: bold;
        text-transform: uppercase;
        letter-spacing: 1px;
        transition: background-color 0.3s ease;
      }
      input[type="submit"]:hover {
        background-color: #0c85d0;
      }
    </style>
  </head>
  <body>
    <div class="form-container">
      <h1>Text Classification Demo</h1>
      <form method="POST" action="{{ url_for('process_form') }}">
        <label for="text">Enter some text:</label>
        <input type="text" name="text" id="text" placeholder="Type your text here">
        <input type="submit" value="Predict">
      </form>
    </div>
  </body>
</html>
"""

results_html = """<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <meta http-equiv="X-UA-Compatible" content="ie=edge">
    <title>Result</title>
    <style>
        body {
            background-color: #15202B;
            color: #fff;
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh;
            flex-direction: column;
            font-family: Arial, Helvetica, sans-serif;
        }

        h1 {
            font-size: 3rem;
            margin: 1rem 0;
            color: #1DA1F2;
        }

        p {
            font-size: 1.5rem;
            margin: 1rem 0;
            color: #1DA1F2;
            font-weight: bold;
        }

        a {
            color: #1DA1F2;
            text-decoration: none;
            font-weight: bold;
            margin-top: 2rem;
        }

        a:hover {
            text-decoration: underline;
        }

        .container {
            display: flex;
            flex-direction: column;
            align-items: center;
        }

        .result {
            margin-top: 3rem;
            font-size: 3rem;
            color: green;
        }
    </style>
</head>
<body>
    <div class="container">
        <h1>Result</h1>
        <p>The prediction is:</p>
        <div class="result">{{ prediction }}</div>
        <a href="/">Return to main page</a>
    </div>
</body>
</html>
"""

with open('templates/index.html','w') as f:
    f.write(index_html)

with open('templates/results.html','w') as f:
    f.write(results_html)


def clean_text(text):
    #normalize text
    text=araby.normalize_alef(text)
    text=araby.normalize_hamza(text)
    text=araby.strip_tashkeel(text)
    text=araby.strip_diacritics(text)
    text=araby.strip_harakat(text)
    #get just arabic text
    text=re.sub(r'[u0600-u06FF]+', '', text).strip()
    text=re.sub(r'[a-z]+', '', text).strip()
    #remove duplicate letter
    text=re.sub(r'(.)\1+', r'\1', text).strip()
    text=re.sub(r"(.)(ـ)+", r'\1', text).strip()
    #removing symbols
    text=' '.join(w for w in re.split(r"\W", text) if w)
    #remove letters
    text=' '.join(w for w in araby.tokenize(text) if len(w)>1)
    return text

In [4]:
app = Flask(__name__)
run_with_ngrok(app)

# Load the pre-trained model and the vectorizer
model = joblib.load('TF-IDF-model.joblib')

@app.route('/', methods=['GET'])
def index():
    return render_template('index.html')

def predict(text):
    # Make a prediction using the pre-trained model
    text = clean_text(text)
    y_pred = model.predict([text])

    # Return the predicted class as a response
    response = y_pred[0]
    return response

@app.route('/process', methods=['POST'])
def process_form():
    # process the form data here
    text = request.form['text']
    prediction = predict(text)
    # redirect the user to the results page
    return redirect(url_for('results', prediction=prediction))

@app.route('/results/<prediction>')
def results(prediction):
    return render_template('results.html', prediction=prediction)

app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://2dcf-34-133-71-55.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [19/Jun/2023 19:55:45] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Jun/2023 19:55:46] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [19/Jun/2023 19:55:53] "POST /process HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [19/Jun/2023 19:55:53] "GET /results/EG HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Jun/2023 19:55:56] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Jun/2023 19:56:03] "POST /process HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [19/Jun/2023 19:56:03] "GET /results/EG HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Jun/2023 19:56:05] "GET / HTTP/1.1" 200 -
